# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [ ]:
import pandas as pd

def cargar_ventas(archivo):
    ventas = []
    
    with open(archivo, "r") as file:
        lineas = file.readlines()
    
    for linea in lineas:
        fecha = linea[0:10].strip()       
        producto = linea[10:40].strip()   
        precio = float(linea[40:50].strip())  
        cantidad = int(linea[50:55].strip())  
        
        venta = {
            'fecha': fecha,
            'producto': producto,
            'precio': precio,
            'cantidad': cantidad
        }
        
        ventas.append(venta)
    
    df_ventas = pd.DataFrame(ventas)
    
    return df_ventas

archivo = 'ventas.txt'  
df = cargar_ventas(archivo)

print(df.head())

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [ ]:

import pandas as pd

def leer_datos(archivo):
    fechas = []
    productos = []
    precios = []
    cantidades = []
    
    with open(archivo, 'r') as file:
        for line in file:
            line = line.strip()
            if line:  
                partes = line.split()
                fecha = partes[0]  
                producto = " ".join(partes[1:-2])  
                precio = int(partes[-2])  
                cantidad = int(partes[-1])  
                
                fechas.append(fecha)
                productos.append(producto)
                precios.append(precio)
                cantidades.append(cantidad)
    
    df = pd.DataFrame({
        'fecha': fechas,
        'producto': productos,
        'precio': precios,
        'cantidad': cantidades
    })
    
    return df

def calcular_totales(df):
    df['importe'] = df['precio'] * df['cantidad']
    total_importe = df['importe'].sum()
    total_cantidad = df['cantidad'].sum()
    
    return total_importe, total_cantidad

archivo = 'datos.dat'
df_datos = leer_datos(archivo)

importe, cantidad = calcular_totales(df_datos)

print(f"Las ventas fueron de ${importe:,.2f} en {cantidad} unidades")

### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [ ]:
import pandas as pd

def unidades_vendidas(archivo):
    df = pd.read_csv(archivo, delim_whitespace=True, header=None, names=['fecha', 'producto', 'precio', 'cantidad'])
    
    ventas_por_producto = df.groupby('producto')['cantidad'].sum()
    
    return ventas_por_producto

def listar_ventas(ventas):
    for producto, cantidad in ventas.items():
        print(f"Producto: {producto}, Unidades vendidas: {cantidad}")

archivo = 'datos.dat'

ventas = unidades_vendidas(archivo)

listar_ventas(ventas)

###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [ ]:
import pandas as pd

def leer_datos(archivo):
    columnas = ['fecha', 'producto', 'precio', 'cantidad']
    
    try:
        df = pd.read_fwf(archivo, widths=[10, 30, 5, 5], names=columnas, dtype={'precio': 'float', 'cantidad': 'int'})
    except ValueError as e:
        print(f"Error al leer el archivo: {e}")
        return pd.DataFrame()
    
    df = df.dropna()
    
    df = df[(df['precio'] > 0) & (df['cantidad'] > 0)]
    
    return df

def precio_promedio(df):
    df['total_importe'] = df['precio'] * df['cantidad']
    precios_promedio = df.groupby('producto').apply(lambda x: x['total_importe'].sum() / x['cantidad'].sum())
    
    return precios_promedio

def listar_precios(precios):
    print("Precio promedio por producto:")
    for producto, precio in precios.items():
        print(f"{producto}: {precio:.2f}")

archivo = 'datos.dat'
df_datos = leer_datos(archivo)

if not df_datos.empty:
    precios = precio_promedio(df_datos)
    listar_precios(precios)
else:
    print("No se pudo procesar el archivo.")

### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [ ]:
Tp3 punto 5 

import pandas as pd

def leer_datos(archivo):
    df = pd.read_csv(archivo, delim_whitespace=True, header=None, names=['producto', 'unidades'])
    
    df['producto'] = df.iloc[:, :-1].agg(' '.join, axis=1)  
    df['unidades'] = df.iloc[:, -1]  
    
    df = df[['producto', 'unidades']]
    
    return df

def ranking_productos(df, top=3):
    ventas = df.groupby('producto')['unidades'].sum().reset_index()
    
    ranking = ventas.sort_values(by='unidades', ascending=False)
    
    return ranking.head(top)

def listar_ranking(ranking):
    for i, (producto, unidades) in enumerate(zip(ranking['producto'], ranking['unidades']), start=1):
        print(f"{i}. {producto}: {unidades} unidades vendidas")

archivo = 'datos.dat'
df_datos = leer_datos(archivo)

if not df_datos.empty:
    ranking = ranking_productos(df_datos)
    listar_ranking(ranking)
else:
    print("No se pudo procesar el archivo.")

### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [ ]:
import pandas as pd

def leer_datos(archivo):
    df = pd.read_csv(archivo, delim_whitespace=True, header=None, names=['fecha', 'producto', 'unidades'])
    
    df['producto'] = df.iloc[:, 1:-1].agg(' '.join, axis=1) 
    df['unidades'] = df.iloc[:, -1]  
    
    df = df[['fecha', 'producto', 'unidades']]
    
    return df

def ventas_por_mes(df):
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes_anio'] = df['fecha'].dt.to_period('M')  
    
    ventas = df.groupby(['mes_anio', 'producto'])['unidades'].sum().unstack(fill_value=0)
    
    return ventas

def listar_ventas_mensuales(ventas):
    for mes, productos in ventas.iterrows():
        print(f"Ventas para el mes: {mes}")
        for nombre, unidades in productos.items():
            print(f"  {nombre}: {unidades} unidades vendidas")
        print() 

archivo = 'datos.dat'
df_datos = leer_datos(archivo)

if not df_datos.empty:
    ventas = ventas_por_mes(df_datos)
    listar_ventas_mensuales(ventas)
else:
    print("No se pudo procesar el archivo.")

### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [ ]:
import pandas as pd
import re

def leer_datos(archivo):
    datos = []
    with open(archivo, 'r') as f:
        for linea in f:
            linea = linea.strip()
            match = re.match(r'(\d{4}-\d{2}-\d{2})(\S+(?:\s+\S+)*)\s+(\d+)\s+(\d+)', linea)
            if match:
                fecha = match.group(1)
                producto = match.group(2)
                precio = int(match.group(3))
                cantidad = int(match.group(4))

                datos.append({'fecha': fecha, 'producto': producto, 'precio': precio, 'cantidad': cantidad})
            else:
                print(f"Formato incorrecto en la línea: {linea}")

    return pd.DataFrame(datos)

def resumen_ventas(df):
    resumen = df.groupby('producto').agg(
        total_vendido=('precio', lambda x: (x * df.loc[x.index, 'cantidad']).sum()),
        cantidad_vendida=('cantidad', 'sum')
    ).reset_index()
    
    resumen['precio_promedio'] = resumen['total_vendido'] / resumen['cantidad_vendida']
    
    return resumen

def informe_ventas(resumen):
    productos_ordenados = resumen.sort_values(by='producto')

    print("Listado de Productos:")
    print(f"{'Producto':<30} {'Precio Promedio':<20} {'Cantidad Vendida':<20} {'Importe Total':<20}")

    for _, row in productos_ordenados.iterrows():
        print(f"{row['producto']:<30} {row['precio_promedio']:<20.2f} {row['cantidad_vendida']:<20} {row['total_vendido']:<20.2f}")

archivo = 'datos.dat'
df_datos = leer_datos(archivo)

if not df_datos.empty:
    resumen = resumen_ventas(df_datos)
    informe_ventas(resumen)
else:
    print("No se pudo procesar el archivo.")

## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [4]:
import numpy as np
def f(x, coeficientes):
    a,b,c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return y - y_pred

X = np.array([0,1,2,3,5])
Y = np.array([0,8,12,12,0])

def buscar_coeficientes():
    pass # Implementar

coeficientes = buscar_coeficientes()
coeficientes

(-2, 10, 0)

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [ ]:
import numpy as np

puntos = [(0, 0), (1, 8), (2, 12), (3, 12), (5, 0)]

def f(x, a, b, c):
    return a*x**2 + b*x + c

def error_total(a, b, c, puntos):
    error = 0
    for x, y in puntos:
        error += abs(f(x, a, b, c) - y)
    return error

coeficientes = np.random.randint(-10, 10, 3)
a, b, c = coeficientes

error = error_total(a, b, c, puntos)
print(f"Coeficientes iniciales: a = {a}, b = {b}, c = {c}")
print(f"Error inicial: {error}")

aprendizaje = 0.001

while error > 1:
    nuevos_coeficientes = coeficientes + np.random.uniform(-aprendizaje, aprendizaje, 3)
    nuevo_a, nuevo_b, nuevo_c = nuevos_coeficientes
    
    nuevo_error = error_total(nuevo_a, nuevo_b, nuevo_c, puntos)
    
    if nuevo_error < error:
        coeficientes = nuevos_coeficientes
        a, b, c = coeficientes
        error = nuevo_error
    
    print(f"Coeficientes actuales: a = {a}, b = {b}, c = {c}")
    print(f"Error actual: {error}")

print(f"\nCoeficientes finales: a = {a}, b = {b}, c = {c}")
print(f"Error final: {error}")

for x, y in puntos:
    print(f"f({x}) = {f(x, a, b, c):.4f}, valor esperado = {y}")


: 